<a href="https://colab.research.google.com/github/GustavinhoDantas/LeituraPDFpy/blob/main/LeitorPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LEITOR DE PDF**
A ideia é pegar o PDF e ler de maneira automatica, onde o valor lidro ira retornar como arquivo excel ou SQL

In [2]:
!pip install pdfplumber pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 86.2 MB/s eta 0:00:00


In [9]:
import pdfplumber
import re
import pandas as pd

# Lista para armazenar os dicionários dos dados extraídos
dados = []

# Função que extrai o valor de um campo no formato "Campo.......: valor"
def extrair_valor(campo, texto):
    # regex para capturar depois do "campo" até quebra de linha ou próximo campo
    if campo == 'Placa.......':
      pattern = re.compile(rf'{re.escape(campo)}\.*:\s*(.{7})')  # Pega 7 caracteres após o campo
      match = pattern.search(texto)
      if match:
        return match.group(1).strip()

    if campo == 'Marca':
        pattern = re.compile(rf'{re.escape(campo)}\.*:\s*([^A]*)')  # Pega até o primeiro 'A'
        match = pattern.search(texto)
        if match:
            return match.group(1).strip()

    # Caso não seja Placa nem Marca, aplica o padrão normal
    pattern = re.compile(rf'{re.escape(campo)}\.*:\s*(.*)')
    match = pattern.search(texto)
    if match:
        return match.group(1).strip()

    return ''


with pdfplumber.open('/content/Leo_Teste_PDF.pdf') as pdf:
    texto_completo = ''
    for pagina in pdf.pages:
        texto_completo += pagina.extract_text() + '\n'

    # Quebra o texto em blocos usando a linha cheia de pontos
    blocos = re.split(r'\n\.{10,}\n', texto_completo)

    for bloco in blocos:
        # Só processar blocos que tem "Placa.......:"
        if 'Placa.......:' in bloco:
            # Extrai os campos desejados de forma simples
            placa = extrair_valor('Placa.......', bloco)
            marca = extrair_valor('Marca', bloco)
            proprietario = extrair_valor('Proprietário Atual', bloco)
            telefone_resp = extrair_valor('Telefone Responsável', bloco)
            origem = extrair_valor('Origem', bloco)
            destinatario = extrair_valor('Destinatário', bloco)
            chassi = extrair_valor('Chassi', bloco)
            ano_fabricado = extrair_valor('Ano Fabricado', bloco)
            ano_modelo = extrair_valor('Ano Modelo', bloco)
            telefone_comercial = extrair_valor('Telefone Comercial', bloco)
            exercicio = extrair_valor('Exercício', bloco)
            renavam = extrair_valor('Renavam', bloco)
            telefone_celular = extrair_valor('Telefone Celular', bloco)

            dados.append({
                'PLACA': placa,
                'MARCA': marca,
                'PROPRIETÁRIO ATUAL': proprietario,
                'TELEFONE RESPONSÁVEL': telefone_resp,
                'ORIGEM': origem,
                'DESTINATÁRIO': destinatario,
                'CHASSI': chassi,
                'ANO FABRICADO': ano_fabricado,
                'ANO MODELO': ano_modelo,
                'TELEFONE COMERCIAL': telefone_comercial,
                'EXERCÍCIO': exercicio,
                'RENAVAM': renavam,
                'TELEFONE CELULAR': telefone_celular
            })

# Transforma em DataFrame e salva
df = pd.DataFrame(dados)
df.to_excel('dados_extraidos.xlsx', index=False)
print('Dados extraídos e salvos no Excel!')


Dados extraídos e salvos no Excel!
